## libs

In [1]:
import sys
import os

src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
import logging
from src.core.llm_generators.specification import AnalyticsSpecGenerator
from src.core.llm_generators.airflow import AirflowDagGenerator
from src.core.llm_generators.dbt import DbtGenerator
from src.core.llm_generators.dashboards import MetabaseDashboardGenerator
from src.config.prompts import prompts
from src.config.settings import settings

In [3]:
logging.basicConfig(level=logging.INFO, filename="exp1_log.log",filemode="w")

## 1 ШАГ: Описание / Генерация ТЗ

In [4]:
user_description = """
Необходимо построить аналитическую систему для мониторинга погодных условий в Москве. 
Предполагается, что аналитикой будут пользоваться менеджеры.

Источником данных является OpenMeteo API.
Требуется в 30 минут каждого часа (например, в 13:30, 14:30 и т.д.) собирать информацию о температуре воздуха, влажности, скорости ветра и осадках. 

На основе этих данных нужно рассчитывать среднесуточные значения, а также выявлять дни с экстремальными погодными условиями (например, сильный ветер или аномально высокая температура). 
Итоговый дашборд должен отображать динамику погодных параметров и выделять аномалии."
"""

In [5]:
spec_gen = AnalyticsSpecGenerator()

In [6]:
result = spec_gen.extract_info_from_users_desription(user_description)

In [7]:
spec = result

## 2 ШАГ: Генерация Airflow DAG

In [8]:
airflow_gen = AirflowDagGenerator(analytics_specification=spec)

In [9]:
airflow_gen.generate_dag()

In [22]:
import requests
base_url = f'https://api.open-meteo.com/v1/forecast'

# Дата выполнения DAG
execution_date = "2025-05-26"

# Формирование запроса к API
params = {
    'latitude': 55.7558,  # Координаты Москвы
    'longitude': 37.6173,
    'start_date': execution_date,
    'end_date': execution_date,
    'hourly': ','.join(["temperature_2m"]),
}

response = requests.get(base_url, params=params)
response.raise_for_status()

# Извлечение данных из ответа API
data = response.json().get('hourly', {})
records = [
    {
        'timestamp': f"{execution_date}T{hour}:00:00",
        'temperature': data['temperature_2m'][hour],
        #'humidity': data['humidity'][hour],
        #'wind_speed': data['wind_speed'][hour],
        #'precipitation': data['precipitation'][hour]
    }
    for hour in range(len(data.get('time', [])))
]

In [23]:
records

[{'timestamp': '2025-05-26T0:00:00', 'temperature': 14.6},
 {'timestamp': '2025-05-26T1:00:00', 'temperature': 13.9},
 {'timestamp': '2025-05-26T2:00:00', 'temperature': 13.6},
 {'timestamp': '2025-05-26T3:00:00', 'temperature': 14.5},
 {'timestamp': '2025-05-26T4:00:00', 'temperature': 16.5},
 {'timestamp': '2025-05-26T5:00:00', 'temperature': 19.1},
 {'timestamp': '2025-05-26T6:00:00', 'temperature': 21.7},
 {'timestamp': '2025-05-26T7:00:00', 'temperature': 24.3},
 {'timestamp': '2025-05-26T8:00:00', 'temperature': 26.2},
 {'timestamp': '2025-05-26T9:00:00', 'temperature': 27.0},
 {'timestamp': '2025-05-26T10:00:00', 'temperature': 27.7},
 {'timestamp': '2025-05-26T11:00:00', 'temperature': 28.0},
 {'timestamp': '2025-05-26T12:00:00', 'temperature': 28.2},
 {'timestamp': '2025-05-26T13:00:00', 'temperature': 28.0},
 {'timestamp': '2025-05-26T14:00:00', 'temperature': 27.7},
 {'timestamp': '2025-05-26T15:00:00', 'temperature': 27.0},
 {'timestamp': '2025-05-26T16:00:00', 'temperature

## 3 ШАГ: Генерация DBT

In [10]:
dbt_llm = DbtGenerator(analytics_specification=spec)

#dbt_llm._generate_sources()
dbt_llm.fill_and_save_project()